# Materials lecture

In [61]:
import hvplot.pandas # make plots from pandas using the same API
import hvplot
pn.extension() #use the jupyter notebook to show interactive plots

import pandas as pd
from sklearn.ensemble import HistGradientBoostingRegressor

import panel as pn

palette = ["#3d348b","#e6af2e","#191716","#a65628"] #use your favourite colours

In [44]:
# Create data with x and y variables
def create_data(N=1000):
    import numpy as np
    x = np.sort(np.random.randn(N))
    
    # Sample data
    df = pd.DataFrame({
        'x': x,
        'y': x+np.random.randn(N),
        'class': ["class A"]*int(N/2) + ["class B"]*int(N/2)
    })

    return df


# 1. hvPlot

## Basic usage

In [267]:
#show how the data looks like
df.sample(n=10)

,x,y,class
66,0.531270,-1.485598,class B
71,0.667643,1.531489,class B
19,-0.938824,-0.161919,class A
56,0.309595,1.878261,class B
64,0.500145,1.153726,class B
25,-0.699652,-0.001102,class A
63,0.456102,0.584033,class B
98,2.206213,2.246861,class B
83,1.228943,0.074922,class B
53,0.223755,0.614522,class B


In [268]:
# Create data
df = create_data(100)

# Plot
scatter = df.hvplot.scatter(x='x',
                            y='y',
                            xlabel="Var x",
                            ylabel="Var y",
                           )

# Show
scatter

:Scatter   [x]   (y)

In [269]:
# Fit a fancy model on the data
fancy_model = HistGradientBoostingRegressor().fit(df[["x"]], df["y"])
df["pred"] = fancy_model.predict(df[["x"]])

# Plot the predicted values (careful, this is in-sample)
s = df.hvplot.line(x='x',
                   y='pred',
                   color="gray"
                   )
                           
# Show
scatter * s

:Overlay
   .Scatter.I :Scatter   [x]   (y)
   .Curve.I   :Curve   [x]   (pred)

In [73]:
# Plot, coloring by class
scatter = df.hvplot.scatter(x='x',
                            y='y',
                            by="class",
                            xlabel="Var x",
                            ylabel="Var y",
                            legend="top_left"
                           )

# Show
scatter

:NdOverlay   [class]
   :Scatter   [x]   (y)

In [84]:
# Plot, grouping by class
scatter = df.hvplot.scatter(x='x',
                            y='y',
                            groupby="class",
                            xlabel="Var x",
                            ylabel="Var y",
                            legend="top_left",
                           )

# Show
scatter

:DynamicMap   [class]
   :Scatter   [x]   (y)

In [94]:
#10 M plot
df = create_data(10000000)

# Plot
large_scatter = df.hvplot.scatter(x='x',
                            y='y',
                            xlabel="Var x",
                            ylabel="Var y",
                            rasterize=True
                           )

# Show
large_scatter

:DynamicMap   []
   :Image   [x,y]   (x_y Count)

In [126]:
#
df = create_data(100)

# Plot
scatter = df.hvplot.scatter(x='x',
                            y='y',
                            xlabel="Var x",
                            ylabel="Var y",
                            legend="top_left",
                           )

# Show
scatter.opts(show_grid=True,
            toolbar="below")

:Scatter   [x]   (y)

# 2. Panel

## 2.1 Panes
Pane objects allow wrapping external viewable items like Bokeh, Plotly, Vega, or HoloViews plots, so they can be embedded in a panel.

In [93]:
png_pane = pn.pane.Image('https://assets.holoviz.org/panel/samples/png_sample.png', width=400)

png_pane

Image(str, width=400)

## 2.2 Layout
Layouts allow to organize the panes

In [127]:
# Combine plots Linked by default!!
pn.Row(large_scatter, scatter)

Row
    [0] HoloViews(DynamicMap, height=300, sizing_mode='fixed', width=700)
    [1] HoloViews(Scatter, height=300, sizing_mode='fixed', width=700)

## 2.3 Templates
Templates provide pre-made layouts

In [202]:
# Instantiate the template with widgets displayed in the sidebar
template = pn.template.BootstrapTemplate(
    sidebar="Here we would usually have some controls",
    main=[pn.Column(large_scatter, scatter, sizing_mode="scale_width")]
)


template.show()

Launching server at http://localhost:53242


In [211]:
explanation = """
# Welcome to this cool app

This app shows my random plots
"""
explanation = pn.pane.Markdown(explanation)


# Instantiate the template with widgets displayed in the sidebar
template = pn.template.MaterialTemplate(
    title='Example template',
    sidebar=[explanation, png_pane],
    main=[pn.Column(large_scatter, scatter)],
    header=pn.pane.Str("This is the header"),
    header_background="#A01346" ,
    
)

template.show()


Launching server at http://localhost:62958


In [212]:
# Data gets embedded
template.save("apps/lecture_app.html")

# 3. Interactivity/Reactivity

## 3.1 Using data.interactive()

In [235]:
# Create a widget
w_select_class = pn.widgets.Select(options=df["class"].unique().tolist())

# Make the data interactive (a perk of hvplot!)
dfi = df.interactive()

# Filter and visualize the data (the parenthesis define the pipe)
(dfi
    .loc[dfi["class"] == w_select_class]
    .hvplot("x", 
            "y", 
            kind="scatter", 
            xlim=(-3,3), 
            ylim=(-6,6)
           )
)

## 3.2 Using pn.bind()

In [242]:
w_select_class = pn.widgets.Select(options=df["class"].unique().tolist())

def interactive_plot(class_subset):
    return (
        df
        .loc[df["class"] == class_subset]    
        .hvplot("x", 
                "y", 
                kind="scatter", 
                xlim=(-3,3), 
                ylim=(-6,6)
               )
    )

interactive_plot2 = pn.bind(interactive_plot, class_subset=w_select_class)

pn.Column(w_select_class, interactive_plot2)

Column
    [0] Select(options=['class A', 'class B'], value='class A')
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False)

In [295]:
# Widgets
operation_selector = pn.widgets.Select(name="Operation", options=['Addition', 'Subtraction', 'Multiplication', 'Division'])
column_selector1 = pn.widgets.Select(name="First Column", options=["x", "y"])
column_selector2 = pn.widgets.Select(name="Second Column", options=["x", "y"])

def plot_data(operation, col1, col2):
    if operation == 'Addition':
        result = df[col1] + df[col2]
    elif operation == 'Subtraction':
        result = df[col1] - df[col2]
    elif operation == 'Multiplication':
        result = df[col1] * df[col2]
    elif operation == 'Division':
        result = data[col1] / data[col2]
    return result.hvplot(kind="scatter", xlabel="Row number", ylabel=f"{operation} of {col1} and {col2}")

# Bind the function to widget values
interactive_plot = pn.bind(plot_data, operation=operation_selector, col1=column_selector1, col2=column_selector2)

# Display
dashboard = pn.Column(operation_selector, column_selector1, column_selector2, interactive_plot)
dashboard.servable()


Column
    [0] Select(name='Operation', options=['Addition', 'Subtraction'...], value='Addition')
    [1] Select(name='First Column', options=['x', 'y'], value='x')
    [2] Select(name='Second Column', options=['x', 'y'], value='x')
    [3] ParamFunction(function, _pane=HoloViews, defer_load=False)

## 3.3 Using buttons

In [266]:
w_button = pn.widgets.Button(name='Click me', button_type='primary')
selections = pn.pane.Markdown(object='You have not clicked the button')    

def print_selected(event):
    selections.object = f"You have clicked the button {event.new} times"
    

w_button.on_click(print_selected)

pn.Column(w_button, selections)

Column
    [0] Button(button_type='primary', name='Click me')
    [1] Markdown(str)

## 3.4 Using widget.link() or widget.param.watch()


### 3.4.1 .link()

.link is primarily useful for creating direct one-to-one connections between widget properties. With .link, you can directly connect one widget's property to another without any intermediate computation or processing.

For instance, if you want two sliders to always have the same value, or if you want a toggle button to directly control the visibility of another widget, .link is straightforward and ideal.

This kind of direct linkage is not what pn.bind is designed for. pn.bind is more about creating bound functions that get re-evaluated when input widget values change.

In [299]:
# Linking a pane to a widgget
w_select_class = pn.widgets.Select(options=df["class"].unique().tolist())
selections = pn.pane.Markdown(object=f'Select a class')    


w_select_class.link(selections, value="object")

pn.Column(w_select_class, selections)

Column
    [0] Select(options=['class A', 'class B'], value='class A')
    [1] Markdown(str)

In [304]:
# Linking two widgets

toggle = pn.widgets.Switch(name='Show Textbox', value=True)
textbox = pn.widgets.TextInput(name='My Textbox', value='Hello, Panel!')

toggle.link(textbox, value='visible')

pn.Column(toggle, textbox)

Column
    [0] Switch(name='Show Textbox', value=True)
    [1] TextInput(name='My Textbox', value='Hello, Panel!')

### 3.4.2 .watch()

.watch() "watches" for changes in the widget, and then activates a function

Is more flexible, but can make the code more complicated


In [314]:
# Linking a pane to a widgget
w_select_class = pn.widgets.Select(options=df["class"].unique().tolist())
selections = pn.pane.Markdown(object=f'Select a class')    

def selected_class(event):
    selections.object = f"The selected class is {event.new}"

w_select_class.param.watch(selected_class, "value")

pn.Column(w_select_class, selections)

Column
    [0] Select(options=['class A', 'class B'], value='class A')
    [1] Markdown(str)

In [315]:
# You can also do it with pn.bind
w_select_class = pn.widgets.Select(options=df["class"].unique().tolist())

# Define a function that returns the Markdown text based on the selected class
def update_selection(selected):
    return f"The selected class is {selected}"

# Bind the function to the widget value
selections = pn.pane.Markdown(object=pn.bind(update_selection, w_select_class))

pn.Column(w_select_class, selections)


Column
    [0] Select(options=['class A', 'class B'], value='class A')
    [1] Markdown(str)

In [218]:
# You can take a look at the controls of a widget this way
d = pn.widgets.FileInput()
pn.Row(d.controls(), d)

Row
    [0] Tabs
        [0] WidgetBox(margin=(5, 10), name='Controls')
            [0] StaticText(value='<b>Controls</b>')
            [1] Checkbox(name='Disabled')
            [2] TextInput(name='Accept')
            [3] TextInput(description='An HTML string d..., name='Description')
            [4] LiteralInput(name='Mime type', serializer='json')
            [5] Checkbox(name='Multiple')
            [6] LiteralInput(name='Value', serializer='json')
        [1] WidgetBox(margin=(5, 10), name='Layout')
            [0] StaticText(value='<b>Layout</b>')
            [1] LiteralInput(description='Whether the object should..., name='Align', serializer='json', value='start')
            [2] LiteralInput(description='Describes the proportiona..., name='Aspect ratio', serializer='json')
            [3] ListInput(description='CSS classes t..., name='Css classes', serializer='json', type=<class 'list'>)
            [4] IntInput(description='The height of the compone..., name='Height', start=0)
            [5] IntInput(description='Minimal width o..., name='Min width', start=0)
            [6] IntInput(description='Minimal height o..., name='Min height', start=0)
            [7] IntInput(description='Minimal width o..., name='Max width', start=0)
            [8] IntInput(description='Minimal height o..., name='Max height', start=0)
            [9] LiteralInput(description='Allows to create addition..., name='Margin', serializer='json', value=(5, 10))
            [10] DictInput(description='Dictionary of C..., name='Styles', serializer='json', type=<class 'dict'>)
            [11] ListInput(description='List of arbitrary t..., name='Tags', serializer='json', type=<class 'list'>)
            [12] IntInput(description='The width of the componen..., name='Width', start=0)
            [13] Select(description='Describes how the compone..., name='Width policy', options=OrderedDict([('auto', ...]), value='auto')
            [14] Select(description='Describes how the compone..., name='Height policy', options=OrderedDict([('auto', ...]), value='auto')
            [15] Select(description='How the component should ..., name='Sizing mode', options=OrderedDict([('fixed', ...]))
            [16] Checkbox(name='Visible', value=True)
    [1] FileInput()

# 4. Exporting apps

In [289]:
# Create a widget
w_select_class = pn.widgets.Select(options=df["class"].unique().tolist())

# Make the data interactive (a perk of hvplot!)
dfi = df.interactive()

# Filter and visualize the data (the parenthesis define the pipe)
## IMPORTANT: The OUTPUT() will give only the plot, which you join with the widget later. Otherwise it won't export well
viz = (dfi
    .loc[dfi["class"] == w_select_class]
    .hvplot("x", 
            "y", 
            kind="scatter", 
            xlim=(-3,3), 
            ylim=(-6,6),
            responsive=True #if you want it to be responsive
           ).output()
)

app = pn.Column(w_select_class, viz)

# EMbedd saves the different states (careful!)
app.save("apps/viz_with_widget.html", embed=True)